In [ ]:
import os
import torch
import json
import numpy as np
from matplotlib.pyplot import imshow
from PIL import Image, ImageDraw
import pickle

In [ ]:
root = "/home/username/source/SRD/"
sup_file = root + "datasets/gqa/gqa_sup_data.pk"
new_rel_file_1 = root + "output/GQA/relabel-transformer/N50/relation_cache_transformer_pretrain_idf_N50_CD-F15-o1.npy"
new_rel_file_2 = root + "output/GQA/relabel-transformer/N100/relation_cache_transformer_pretrain_idf_N100_CD.npy"
info_file = root + "datasets/gqa/GQA_200_ID_Info.json"

out_dir = root + "output/vis_gqa/"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
info = json.load(open(info_file, 'r'))
from maskrcnn_benchmark.data.datasets.gqa import load_info
ind_to_classes, ind_to_predicates = load_info(info_file)

sup_data = pickle.load(open(sup_file, "rb"))
new_rel_data_n50 = np.load(new_rel_file_1, allow_pickle=True)[()]
new_rel_data_n100 = np.load(new_rel_file_2, allow_pickle=True)[()]

In [ ]:
from collections import Counter

gt_rel_1 = []
for i in range(len(sup_data)):
    gt_rels = sup_data[i]["triples"]
    if gt_rels is not None and len(gt_rels) != 0:
        gt_rel_1.extend(gt_rels[:, 2])

print(len(gt_rel_1))

count_gt = Counter(gt_rel_1)

# print(count_gt)

# for k, v in count_gt.items():
#     print(f"{k}-{ind_to_predicates[k]}: {v}")

for i in range(1, len(ind_to_predicates)):
    print(f"{count_gt[ind_to_predicates[i]]}")

In [ ]:
# statistics
from collections import Counter


new_rel_n100 = []
new_rel_n50 = []
for i in range(len(new_rel_data_n100)):
    if new_rel_data_n100[i] is not None:
        new_rel_n100.extend(new_rel_data_n100[i][:, 2])
for i in range(len(new_rel_data_n50)):
    if new_rel_data_n50[i] is not None:
        new_rel_n50.extend(new_rel_data_n50[i][:, 2])
print(len(new_rel_n100), len(new_rel_n50))

n100_count = Counter(new_rel_n100)
n50_count = Counter(new_rel_n50)

# print(n50_count)

# for k, v in n50_count.items():
#     print(f"{k}-{ind_to_predicates[k]}: {v}")


for i in range(1, len(ind_to_predicates)):
    print(f"{n50_count[ind_to_predicates[i]]}")

In [ ]:
freq_rels = np.arange(len(ind_to_predicates))
for r in freq_rels:
    print(ind_to_predicates[r])

In [ ]:
# get image info by index
def get_info_by_idx(idx, new_rel_data):
    sup_data_dict = sup_data[idx]
    new_rel = new_rel_data[idx]
    # if new_rel is not None:
    #     print(idx, sup_data_dict["image_id"])
    image_id = sup_data_dict["image_id"]
    img_path = sup_data_dict["img_path"]
    box = sup_data_dict["boxes"]
    labels = sup_data_dict["labels"]
    names = [ind_to_classes[ii] for ii in labels]
    gt_rels = sup_data_dict["triples"]
    relation = [[str(triple[0])+"-"+names[triple[0]], ind_to_predicates[triple[2]], str(triple[1])+"-"+names[triple[1]]] for triple in gt_rels]
    if new_rel is not None:
        new_rels = [[str(triple[0])+"-"+names[triple[0]], ind_to_predicates[triple[2]], str(triple[1])+"-"+names[triple[1]]] for triple in new_rel]
    else:
        new_rels = []

    w, h = sup_data_dict['width'], sup_data_dict['height']

    boxes = torch.from_numpy(box).reshape(-1, 4)
    return img_path, boxes, names, relation, new_rels, image_id

In [ ]:
def draw_single_box(pic, box, color='red', draw_info=None):
    draw = ImageDraw.Draw(pic)
    x1,y1,x2,y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])
    draw.rectangle(((x1, y1), (x2, y2)), outline=color)
    if draw_info:
        draw.rectangle(((x1, y1), (x1+50, y1+10)), fill=color)
        info = draw_info
        draw.text((x1, y1), info)

def print_list(name, input_list):
    for i, item in enumerate(input_list):
        print(name + ' ' + str(i) + ': ' + str(item))
    
def draw_image(img_path, boxes, labels, gt_rels, pred_rels, image_id, print_img=True):
    img_path = root + img_path
    pic = Image.open(img_path)
    num_obj = boxes.shape[0]
    for i in range(num_obj):
        info = str(i) + "-" + labels[i]
        draw_single_box(pic, boxes[i], draw_info=info)
    if print_img:
        display(pic)
    if print_img:
        # print('*' * 50)
        # print_list('gt_boxes', labels)
        print('*' * 50)
        print_list('gt_rels', gt_rels)
        print('*' * 50)
    print_list('new_rels', pred_rels[:20])
    print('*' * 50)
    # pic.save(os.path.join(out_dir, str(image_id)+'.png'))
    return None

In [ ]:
def show_selected(idx_list):
    for select_idx in idx_list:
        # print(select_idx)
        draw_image(*get_info_by_idx(select_idx, new_rel_data_n100))
        
def show_all(start_idx, length):
    for cand_idx in range(start_idx, start_idx+length):
        print(cand_idx)
        draw_image(*get_info_by_idx(cand_idx, new_rel_data_n100))
        print('*' * 50)
        print("N50")
        draw_image(*get_info_by_idx(cand_idx, new_rel_data_n50), print_img=False)

In [ ]:
show_all(start_idx=0, length=10)
# show_selected([0, 50, 400])
# good example: 50,301,400,
# issue example: 255

In [ ]:
# vis non-empty new relations
for ii in range(10, 20):
    if new_rel_data_n100[ii] is not None and new_rel_data_n50[ii] is not None:
        # print(new_rel_data_n100[ii].shape[0], new_rel_data_n50[ii].shape[0])
        if new_rel_data_n100[ii].shape[0] > new_rel_data_n50[ii].shape[0]:
            print(ii)
            draw_image(*get_info_by_idx(ii, new_rel_data_n100))
            print('*' * 50)
            print("N50")
            draw_image(*get_info_by_idx(ii, new_rel_data_n50), print_img=False)